<a href="https://colab.research.google.com/github/amarabuco/deeplearning-2024.2/blob/main/RNN_From_Scratch_Swear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aprendiz de artistas

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
# DATA_SOURCE_MAPPING = 'dinosaur-island:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F81348%2F189031%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240930%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240930T222544Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D93e04616cb3d9a39b0423d9c61404532b07b5e6af177cc7b1a25ecb72e23e3a096845be17b4760b721928bbc6ea0ad13673694f05bc30110ec4cdc0c072a1a2ac13bf22bbaf4d35782471d95c1319c6d3a615ff2c3e5f8f7a42be366b9e8349b3c56f901ac881180dfaf01360447f1537be12fcdd152ca2ca1dd4d1972c2b24670a5d7514a790f1d7b76c264bbd700c0b81692702272cb8a024c628e53b2de330e82efa6331a89a667fa3a6b7fae4cae1369262a5a4ace66c8189a2de0ba6c5391019a7836e7664958923488ccc3d232bc427e7ba86222f976d8706b482e51aecefdecbdd352c74819a5a4b197ba2c7e68fd2cd3506bf53201005580324ee7c8'

DATA_SOURCE_MAPPING = 'swear-words:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F9071%2F12641%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241003%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241003T114822Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db3817f9b96c9f7ca5024a0ccf9cc4e19dd7a9bb3819b83ce9f661e78e58dc681c3e665aea40539d189df52bf671c1ad026e7d17e432b9db7f83e565dc4e5b85a3571091dc285142e165e53314eb4d6129ade5b5314a4041c2ccf068a8ed37a8b72b2c14ec27c1ded1190be881fd95dbe20bd16a363597abacaa67974856915efb88ea379a29596b184252911f13c4f60033c1157420b2d12c88437518078aff9d4e8ac8383370501247a1d5add856921fecdfe70c2f699cb20d2ee19077af482d406f5132450d76ee6a5a9b727aaff9fdd602f324331ebf07732ba114c3e2654de638bf7bbc99025e877eaf75c105bea3f7961905dd1e8c56c7af26a2a0498f0'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1534 bytes downloaded
Downloaded and uncompressed: swear-words
Data source import complete.


In [3]:
open('/kaggle/input/swear-words/badwords.txt', 'r').read().split('\n')[:10] # Dataset é um conjunto de nomes de palavroes


['4r5e', '5h1t', '5hit', 'a55', 'anal', 'anus', 'ar5e', 'arrse', 'arse', 'ass']

<div style="padding:10px;
            color:#FF9F00;
            margin:10px;
            font-size:150%;
            display:fill;
            border-radius:1px;
            border-style: solid;
            border-color:#FF9F00;
            background-color:#3E3D53;
            overflow:hidden;">
    <center>
        <a id='top'></a>
        <b>Table of Contents</b>
    </center>
    <br>
    <ul>
        <li>
            <a href="#1">1 -  Overview and Imports</a>
        </li>
        <li>
            <a href="#2">2 - Data Preparation</a>
        </li>
        <li>
            <a href="#3">3 - RNN Implementation</a>
        <li>
            <a href="#4">4 - Thank you</a>
        </li>
        <li>
            <a href="#5">5 - References</a>
        </li>
    </ul>
</div>



    
 <a id="1"></a>
<h1 style='background:#FF9F00;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #3E3D53;'>Overview and Imports</center></h1>

# Overview and Imports
**Recurrent Neural Networks (RNNs) are a type of neural network that are designed to handle sequential data, such as time series or natural language text. They achieve this by using a hidden state that is updated for each time step of the input sequence, allowing the network to maintain a memory of previous inputs.**


**This notebook contains an implementation of an RNN that can be used for language modeling. The self takes in a sequence of characters and outputs the probability distribution over the next character in the sequence. The network is trained on a corpus of text and then used to generate new text that has a similar distribution of characters as the training corpus.**

In [4]:
import os
import numpy as np
import scipy as sp
# import cupy as np

 <a id="2"></a>
<h1 style='background:#FF9F00;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #3E3D53;'>Data Preparation</center></h1>
    
# Data Prepartion

In [6]:
class DataGenerator:
    """
    A class for generating input and output examples for a character-level language model.
    """

    def __init__(self, path):
        """
        Initializes a DataGenerator object.

        Args:
            path (str): The path to the text file containing the training data.
        """
        self.path = path

        # Read in data from file and convert to lowercase
        with open(path, encoding='utf-8') as f:
            data = f.read().lower()

        # Create list of unique characters in the data
        self.chars = list(set(data))

        # Create dictionaries mapping characters to and from their index in the list of unique characters
        self.char_to_idx = {ch: i for (i, ch) in enumerate(self.chars)}
        self.idx_to_char = {i: ch for (i, ch) in enumerate(self.chars)}

        # Set the size of the vocabulary (i.e. number of unique characters)
        self.vocab_size = len(self.chars)

        # Read in examples from file and convert to lowercase, removing leading/trailing white space
        with open(path) as f:
            examples = f.readlines()
        self.examples = [x.lower().strip() for x in examples]

    def generate_example(self, idx):
        """
        Generates an input/output example for the language model based on the given index.

        Args:
            idx (int): The index of the example to generate.

        Returns:
            A tuple containing the input and output arrays for the example.
        """
        example_chars = self.examples[idx]

        # Convert the characters in the example to their corresponding indices in the list of unique characters
        example_char_idx = [self.char_to_idx[char] for char in example_chars]

        # Add newline character as the first character in the input array, and as the last character in the output array
        X = [self.char_to_idx['\n']] + example_char_idx
        Y = example_char_idx + [self.char_to_idx['\n']]

        return np.array(X), np.array(Y)


<a id="3"></a>
<h1 style='background:#FF9F00;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #3E3D53;'>RNN Implementation</center></h1>


# RNN Implementation


## RNN  Implementation <a name="3-1"></a>
**The RNN used in this notebook is a basic one-layer RNN. It consists of an input layer, a hidden layer, and an output layer. The input layer takes in a one-hot encoded vector representing a character in the input sequence. This vector is multiplied by a weight matrix  $W_{ax}$ to produce a hidden state vector $a$. The hidden state vector is then passed through a non-linear activation function (in this case, the hyperbolic tangent function) and updated for each time step of the input sequence. The updated hidden state is then multiplied by a weight matrix  $W_{ya}$ to produce the output probability distribution over the next character in the sequence.**

**The RNN is trained using stochastic gradient descent with the cross-entropy loss function. During training, the self takes in a sequence of characters and outputs the probability distribution over the next character. The true next character is then compared to the predicted probability distribution, and the parameters of the network are updated to minimize the cross-entropy loss.**

## Activation Functions
### Softmax Activation Function

**$$\mathrm{softmax}(\mathbf{x})_i = \frac{e^{x_i}}{\sum_{j=1}^n e^{x_j}}
$$**

**The softmax function is commonly used as an activation function in neural networks, particularly in the output layer for classification tasks. Given an input array $x$, the softmax function calculates the probability distribution of each element in the array**




### Tanh Activation
**$$\tanh(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}$$**

**where $x$ is the input to the function. The output of the function is a value between -1 and 1. The tanh activation function is often used in neural networks as an alternative to the sigmoid activation function, as it has a steeper gradient and can better model non-linear relationships in the data.**
****

## Forward propagation:

**During forward propagation, the input sequence is processed through the RNN to generate an output sequence. At each time step, the hidden state and the output are computed using the input, the previous hidden state, and the RNN's parameters.**

**The equations for the forward propagation in a basic RNN are as follows:**

**At time step $t$, the input to the RNN is $x_t$, and the hidden state at time step $t-1$ is $a_{t-1}$. The hidden state at time step $t$ is computed as:**

**$a_t = \tanh(W_{aa} a_{t-1} + W_{ax} x_t + b_a)$**

**where $W_{aa}$ is the weight matrix for the hidden state, $W_{ax}$ is the weight matrix for the input, and $b_a$ is the bias vector for the hidden state.**

**The output at time step $t$ is computed as:**

**$y_t = softmax(W_{ya} a_t + b_y)$**

**where $W_{ya}$ is the weight matrix for the output, and $b_y$ is the bias vector for the output.**
****
## Backward propagation:

**The objective of training an RNN is to minimize the loss between the predicted sequence and the ground truth sequence. Backward propagation calculates the gradients of the loss with respect to the RNN's parameters, which are then used to update the parameters using an optimization algorithm such as Adagrad or Adam.**

**The equations for the backward propagation in a basic RNN are as follows:**

**At time step $t$, the loss with respect to the output $y_t$ is given by:**

**$\frac{\partial L}{\partial y_t} = -\frac{1}{y_{t,i}} \text{ if } i=t_i, \text{ else } 0$**

**where $L$ is the loss function, $y_{t,i}$ is the $i$th element of the output at time step $t$, and $t_i$ is the index of the true label at time step $t$**.

**The loss with respect to the hidden state at time step $t$ is given by:**

**$\frac{\partial L}{\partial a_t} = \frac{\partial L}{\partial y_t} W_{ya} + \frac{\partial L}{\partial h_{t+1}} W_{aa}$**

**where $\frac{\partial L}{\partial a_{t+1}}$ is the gradient of the loss with respect to the hidden state at the next time step, which is backpropagated through time.**

**The gradient with respect to tanh is given by:**
**$\frac{\partial \tanh(a)} {\partial a}$**

**The gradients with respect to the parameters are then computed using the chain rule:**

**$\frac{\partial L}{\partial W_{ya}} = \sum_t \frac{\partial L}{\partial y_t} a_t$**

**$\frac{\partial L}{\partial b_y} = \sum_t \frac{\partial L}{\partial y_t}$**

**$\frac{\partial L}{\partial W_{ax}} = \sum_t \frac{\partial L}{\partial a_t} \frac{\partial a_t}{\partial W_{ax}}$**

**$\frac{\partial L}{\partial W_{aa}} = \sum_t \frac{\partial L}{\partial h_t} \frac{\partial h_t}{\partial W_{aa}}$**

**$\frac{\partial L}{\partial b_a} = \sum_t \frac{\partial L}{\partial a_t} \frac{\partial h_t}{\partial b_a}$**

**where $\frac{\partial h_t}{\partial W_{ax}}$, $\frac{\partial a_t}{\partial W_{aa}}$, and $\frac{\partial h_t}{\partial b_a}$ can be computed as:**

**$\frac{\partial a_t}{\partial W_{ax}} = x_t$**

**$\frac{\partial a_t}{\partial W_{aa}} = a_{t-1}$**

**$\frac{\partial a_t}{\partial b_a} = 1$**

**These gradients are then used to update the parameters of the RNN using an optimization algorithm such as gradient descent, Adagrad, or Adam.**
****
## Loss:

**The cross-entropy loss between the predicted probabilities y_pred and the true targets y_true at a single time step $t$ is:**

**$$H(y_{true,t}, y_{pred,t}) = -\sum_i y_{true,t,i} \log(y_{pred,t,i})$$**

**where $y_{pred,t}$ is the predicted probability distribution at time step $t$, $y_{true,t}$ is the true probability distribution at time step $t$ (i.e., a one-hot encoded vector representing the true target), and $i$ ranges over the vocabulary size.**

**The total loss is then computed as the sum of the cross-entropy losses over all time steps:**

**$$L = \sum_{t=1}^{T} H(y_{true,t}, y_{pred,t})$$**

**where $T$ is the sequence length.**

****

## Train:
**The train method trains the RNN on a dataset using backpropagation through time. The method takes an instance of DataReader containing the training data as input. The method initializes a hidden state vector a_prev at the beginning of each sequence to zero. It then iterates until the smooth loss is less than a threshold value.**

**During each iteration, it retrieves a batch of inputs and targets from the data reader. The RNN then performs a forward pass on the input sequence and computes the output probabilities. The backward pass is performed using the targets and output probabilities to calculate the gradients of the parameters of the network. The Adagrad algorithm is used to update the weights of the network.**

**The method then calculates and updates the loss using the updated weights. The previous hidden state is updated for the next batch. The method prints the progress every 500 iterations by generating a sample of text using the sample method and printing the loss.**


**The train method can be summarized by the following steps:**


**$1.$ Initialize $a_{prev}$ to zero at the beginning of each sequence.**

**$2.$ Retrieve a batch of inputs and targets from the data reader.**

**$3.$ Perform a forward pass on the input sequence and compute the output probabilities.**

**$4.$ Perform a backward pass using the targets and output probabilities to calculate the gradients of the parameters of the network.**

**$5.$ Use the Adagrad algorithm to update the weights of the network.**

**$6.$ Calculate and update the loss using the updated weights.**

**$7.$ Update the previous hidden state for the next batch.**

**$8.$ Print progress every 10000 iterations by generating a sample of text using the sample method and printing the loss.**

**$9.$ Repeat steps $2$-$8$ until the smooth loss is less than the threshold value.**


In [7]:
class RNN:
    """
    A class used to represent a Recurrent Neural Network (RNN).

    Attributes
    ----------
    hidden_size : int
        The number of hidden units in the RNN.
    vocab_size : int
        The size of the vocabulary used by the RNN.
    sequence_length : int
        The length of the input sequences fed to the RNN.
    learning_rate : float
        The learning rate used during training.
    is_initialized : bool
        Indicates whether the AdamW parameters has been initialized.

    Methods
    -------
    __init__(hidden_size, vocab_size, sequence_length, learning_rate)
        Initializes an instance of the RNN class.

    forward(self, X, a_prev)
     Computes the forward pass of the RNN.

    softmax(self, x)
       Computes the softmax activation function for a given input array.

    backward(self,x, a, y_preds, targets)
        Implements the backward pass of the RNN.

   loss(self, y_preds, targets)
     Computes the cross-entropy loss for a given sequence of predicted probabilities and true targets.

    adamw(self, beta1=0.9, beta2=0.999, epsilon=1e-8, L2_reg=1e-4)
       Updates the RNN's parameters using the AdamW optimization algorithm.

    train(self, generated_names=5)
       Trains the RNN on a dataset using backpropagation through time (BPTT).

   predict(self, start)
        Generates a sequence of characters using the trained self, starting from the given start sequence.
        The generated sequence may contain a maximum of 50 characters or a newline character.

    """

    def __init__(self, hidden_size, data_generator, sequence_length, learning_rate):
        """
        Initializes an instance of the RNN class.

        Parameters
        ----------
        hidden_size : int
            The number of hidden units in the RNN.
        vocab_size : int
            The size of the vocabulary used by the RNN.
        sequence_length : int
            The length of the input sequences fed to the RNN.
        learning_rate : float
            The learning rate used during training.
        """

        # hyper parameters
        self.hidden_size = hidden_size
        self.data_generator = data_generator
        self.vocab_size = self.data_generator.vocab_size
        self.sequence_length = sequence_length
        self.learning_rate = learning_rate
        self.X = None

        # model parameters
        self.Wax = np.random.uniform(-np.sqrt(1. / self.vocab_size), np.sqrt(1. / self.vocab_size), (hidden_size, self.vocab_size))
        self.Waa = np.random.uniform(-np.sqrt(1. / hidden_size), np.sqrt(1. / hidden_size), (hidden_size, hidden_size))
        self.Wya = np.random.uniform(-np.sqrt(1. / hidden_size), np.sqrt(1. / hidden_size), (self.vocab_size, hidden_size))
        self.ba = np.zeros((hidden_size, 1))
        self.by = np.zeros((self.vocab_size, 1))

        # Initialize gradients
        self.dWax, self.dWaa, self.dWya = np.zeros_like(self.Wax), np.zeros_like(self.Waa), np.zeros_like(self.Wya)
        self.dba, self.dby = np.zeros_like(self.ba), np.zeros_like(self.by)

        # parameter update with AdamW
        self.mWax = np.zeros_like(self.Wax)
        self.vWax = np.zeros_like(self.Wax)
        self.mWaa = np.zeros_like(self.Waa)
        self.vWaa = np.zeros_like(self.Waa)
        self.mWya = np.zeros_like(self.Wya)
        self.vWya = np.zeros_like(self.Wya)
        self.mba = np.zeros_like(self.ba)
        self.vba = np.zeros_like(self.ba)
        self.mby = np.zeros_like(self.by)
        self.vby = np.zeros_like(self.by)

    def softmax(self, x):
        """
        Computes the softmax activation function for a given input array.

        Parameters:
            x (ndarray): Input array.

        Returns:
            ndarray: Array of the same shape as `x`, containing the softmax activation values.
        """
        # shift the input to prevent overflow when computing the exponentials
        x = x - np.max(x)
        # compute the exponentials of the shifted input
        p = np.exp(x)
        # normalize the exponentials by dividing by their sum
        return p / np.sum(p)

    def forward(self, X, a_prev):
        """
        Compute the forward pass of the RNN.

        Parameters:
        X (ndarray): Input data of shape (seq_length, vocab_size)
        a_prev (ndarray): Activation of the previous time step of shape (hidden_size, 1)

        Returns:
        x (dict): Dictionary of input data of shape (seq_length, vocab_size, 1), with keys from 0 to seq_length-1
        a (dict): Dictionary of hidden activations for each time step, with keys from 0 to seq_length-1
        y_pred (dict): Dictionary of output probabilities for each time step, with keys from 0 to seq_length-1
        """
        # Initialize dictionaries to store activations and output probabilities.
        x, a, y_pred = {}, {}, {}

        # Store the input data in the class variable for later use in the backward pass.
        self.X = X

        # Set the initial activation to the previous activation.
        a[-1] = np.copy(a_prev)
        # iterate over each time step in the input sequence
        for t in range(len(self.X)):
            # get the input at the current time step
            x[t] = np.zeros((self.vocab_size,1))
            if (self.X[t] != None):
                x[t][self.X[t]] = 1
            # compute the hidden activation at the current time step
            a[t] = np.tanh(np.dot(self.Wax, x[t]) + np.dot(self.Waa, a[t - 1]) + self.ba)
            # compute the output probabilities at the current time step
            y_pred[t] = self.softmax(np.dot(self.Wya, a[t]) + self.by)
            # add an extra dimension to X to make it compatible with the shape of the input to the backward pass
         # return the input, hidden activations, and output probabilities at each time step
        return x, a, y_pred

    def backward(self,x, a, y_preds, targets):
        """
        Implement the backward pass of the RNN.

        Args:
        x -- (dict) of input characters (as one-hot encoding vectors) for each time-step, shape (vocab_size, sequence_length)
        a -- (dict) of hidden state vectors for each time-step, shape (hidden_size, sequence_length)
        y_preds -- (dict) of output probability vectors (after softmax) for each time-step, shape (vocab_size, sequence_length)
        targets -- (list) of integer target characters (indices of characters in the vocabulary) for each time-step, shape (1, sequence_length)

        Returns:
        None

        """
        # Initialize derivative of hidden state for the last time-step
        da_next = np.zeros_like(a[0])

        # Loop through the input sequence backwards
        for t in reversed(range(len(self.X))):
            # Calculate derivative of output probability vector
            dy_preds = np.copy(y_preds[t])
            dy_preds[targets[t]] -= 1

            # Calculate derivative of hidden state
            da = np.dot(self.Waa.T, da_next) + np.dot(self.Wya.T, dy_preds)
            dtanh = (1 - np.power(a[t], 2))
            da_unactivated = dtanh * da

            # Calculate gradients
            self.dba += da_unactivated
            self.dWax += np.dot(da_unactivated, x[t].T)
            self.dWaa += np.dot(da_unactivated, a[t - 1].T)

            # Update derivative of hidden state for the next iteration
            da_next = da_unactivated

            # Calculate gradient for output weight matrix
            self.dWya += np.dot(dy_preds, a[t].T)

            # clip gradients to avoid exploding gradients
            for grad in [self.dWax, self.dWaa, self.dWya, self.dba, self.dby]:
                np.clip(grad, -1, 1, out=grad)

    def loss(self, y_preds, targets):
        """
        Computes the cross-entropy loss for a given sequence of predicted probabilities and true targets.

        Parameters:
            y_preds (ndarray): Array of shape (sequence_length, vocab_size) containing the predicted probabilities for each time step.
            targets (ndarray): Array of shape (sequence_length, 1) containing the true targets for each time step.

        Returns:
            float: Cross-entropy loss.
        """
        # calculate cross-entropy loss
        return sum(-np.log(y_preds[t][targets[t], 0]) for t in range(len(self.X)))

    def adamw(self, beta1=0.9, beta2=0.999, epsilon=1e-8, L2_reg=1e-4):
        """
        Updates the RNN's parameters using the AdamW optimization algorithm.
        """
        # AdamW update for Wax
        self.mWax = beta1 * self.mWax + (1 - beta1) * self.dWax
        self.vWax = beta2 * self.vWax + (1 - beta2) * np.square(self.dWax)
        m_hat = self.mWax / (1 - beta1)
        v_hat = self.vWax / (1 - beta2)
        self.Wax -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.Wax)

        # AdamW update for Waa
        self.mWaa = beta1 * self.mWaa + (1 - beta1) * self.dWaa
        self.vWaa = beta2 * self.vWaa + (1 - beta2) * np.square(self.dWaa)
        m_hat = self.mWaa / (1 - beta1)
        v_hat = self.vWaa / (1 - beta2)
        self.Waa -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.Waa)

        # AdamW update for Wya
        self.mWya = beta1 * self.mWya + (1 - beta1) * self.dWya
        self.vWya = beta2 * self.vWya + (1 - beta2) * np.square(self.dWya)
        m_hat = self.mWya / (1 - beta1)
        v_hat = self.vWya / (1 - beta2)
        self.Wya -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.Wya)

        # AdamW update for ba
        self.mba = beta1 * self.mba + (1 - beta1) * self.dba
        self.vba = beta2 * self.vba + (1 - beta2) * np.square(self.dba)
        m_hat = self.mba / (1 - beta1)
        v_hat = self.vba / (1 - beta2)
        self.ba -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.ba)

        # AdamW update for by
        self.mby = beta1 * self.mby + (1 - beta1) * self.dby
        self.vby = beta2 * self.vby + (1 - beta2) * np.square(self.dby)

    def sample(self):
        """
        Sample a sequence of characters from the RNN.

        Args:
            None

        Returns:
            list: A list of integers representing the generated sequence.
        """
        # initialize input and hidden state
        x = np.zeros((self.vocab_size, 1))
        a_prev = np.zeros((self.hidden_size, 1))

        # create an empty list to store the generated character indices
        indices = []

        # idx is a flag to detect a newline character, initialize it to -1
        idx = -1

        # generate sequence of characters
        counter = 0
        max_chars = 50 # maximum number of characters to generate
        newline_character = self.data_generator.char_to_idx['\n'] # the newline character

        while (idx != newline_character and counter != max_chars):
            # compute the hidden state
            a = np.tanh(np.dot(self.Wax, x) + np.dot(self.Waa, a_prev) + self.ba)

            # compute the output probabilities
            y = self.softmax(np.dot(self.Wya, a) + self.by)

            # sample the next character from the output probabilities
            idx = np.random.choice(list(range(self.vocab_size)), p=y.ravel())

            # set the input for the next time step
            x = np.zeros((self.vocab_size, 1))
            x[idx] = 1

            # store the sampled character index in the list
            indices.append(idx)

            # update the previous hidden state
            a_prev = a

            # increment the counter
            counter += 1

        # return the list of sampled character indices
        return indices


    def train(self, generated_names=5):
        """
        Train the RNN on a dataset using backpropagation through time (BPTT).

        Args:
        - generated_names: an integer indicating how many example names to generate during training.

        Returns:
        - None
        """

        iter_num = 0
        threshold = 5 # stopping criterion for training
        smooth_loss = -np.log(1.0 / self.data_generator.vocab_size) * self.sequence_length  # initialize loss

        while (smooth_loss > threshold):
            a_prev = np.zeros((self.hidden_size, 1))
            idx = iter_num % self.vocab_size
            # get a batch of inputs and targets
            inputs, targets = self.data_generator.generate_example(idx)

            # forward pass
            x, a, y_pred  = self.forward(inputs, a_prev)

            # backward pass
            self.backward(x, a, y_pred, targets)

            # calculate and update loss
            loss = self.loss(y_pred, targets)
            self.adamw()
            smooth_loss = smooth_loss * 0.999 + loss * 0.001

            # update previous hidden state for the next batch
            a_prev = a[len(self.X) - 1]
            # print progress every 500 iterations
            if iter_num % 500 == 0:
                print("\n\niter :%d, loss:%f\n" % (iter_num, smooth_loss))
                for i in range(generated_names):
                    sample_idx = self.sample()
                    txt = ''.join(self.data_generator.idx_to_char[idx] for idx in sample_idx)
                    txt = txt.title()  # capitalize first character
                    print ('%s' % (txt, ), end='')
            iter_num += 1

    def predict(self, start):
        """
        Generate a sequence of characters using the trained self, starting from the given start sequence.
        The generated sequence may contain a maximum of 50 characters or a newline character.

        Args:
        - start: a string containing the start sequence

        Returns:
        - txt: a string containing the generated sequence
        """

        # Initialize input vector and previous hidden state
        x = np.zeros((self.vocab_size, 1))
        a_prev = np.zeros((self.hidden_size, 1))

        # Convert start sequence to indices
        chars = [ch for ch in start]
        idxes = []
        for i in range(len(chars)):
            idx = self.data_generator.char_to_idx[chars[i]]
            x[idx] = 1
            idxes.append(idx)

        # Generate sequence
        max_chars = 50  # maximum number of characters to generate
        newline_character = self.data_generator.char_to_idx['\n']  # the newline character
        counter = 0
        while (idx != newline_character and counter != max_chars):
            # Compute next hidden state and predicted character
            a = np.tanh(np.dot(self.Wax, x) + np.dot(self.Waa, a_prev) + self.ba)
            y_pred = self.softmax(np.dot(self.Wya, a) + self.by)
            idx = np.random.choice(range(self.vocab_size), p=y_pred.ravel())

            # Update input vector, previous hidden state, and indices
            x = np.zeros((self.vocab_size, 1))
            x[idx] = 1
            a_prev = a
            idxes.append(idx)
            counter += 1

        # Convert indices to characters and concatenate into a string
        txt = ''.join(self.data_generator.idx_to_char[i] for i in idxes)

        # Remove newline character if it exists at the end of the generated sequence
        if txt[-1] == '\n':
            txt = txt[:-1]

        return txt


In [8]:
data_generator = DataGenerator('/kaggle/input/swear-words/badwords.txt')
rnn = RNN(hidden_size
          =250,data_generator=data_generator, sequence_length=25, learning_rate=1e-3)
rnn.train()



iter :0, loss:92.764947

C0*Pwx11P Mjp*!Je_Lk3.X.Vuaqb7Fbp.V0Sms1Wdkwd31G4QI7 Wi!L
B_+4.Vue5Jfkzb*Vfqs1O-Iw -Vipi-+1Jg1Bti8O.4Ln85Pjj
Xk!Cc1Yoh-D18Ve*Gjr1Eyy


iter :500, loss:64.035684

B1Fttyl
Ghiu
H1Iwr
Siitbts
Najc


iter :1000, loss:44.854968

Asarl
S
Aua
Eh
Rseee


iter :1500, loss:32.990125

Cheaa
+U
As
Hr
4R1-T


iter :2000, loss:25.858365

_Syeee
Selb5
Ys
Asr
Osee


iter :2500, loss:21.318737

Ar
Naiess
_S
Txansayee
Ea


iter :3000, loss:18.463869

A.
Bxlcsdok
0Bsgce
Ch
Ids


iter :3500, loss:16.578099

5H
Np
Gpk
Yp
+P4Ndo


iter :4000, loss:15.197568

Qxmgr
0Dsth
B.Sshesarty
Arsr
K5St


iter :4500, loss:14.309557

*Xf
Bxfc
Yx
Qx
*X


iter :5000, loss:13.381424

Salos
Qd
!Rx
1Xx
H1


iter :5500, loss:12.630043

Qr
*R
5Hxad
Birl
Rx


iter :6000, loss:12.082895

8!Led
Ser
D
D
K5Elrs


iter :6500, loss:11.606411

Irr
Ia1Oolers
Asr
Biljles
Qhn


iter :7000, loss:11.166549

B1Tc
H1D
B1Tch
Chelbcaler
Q5Ee


iter :7500, loss:10.726539

Qct
Qrtol
Ar
_Sfuccc5Ir
Qs


iter :8000, loss:

KeyboardInterrupt: 

Aprendeu a xingar, coisa que brasileiro ensina no exterior 😅

<a id="4"></a>
<h1 style='background:#FF9F00;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #3E3D53;'>Thank you</center></h1>

# Thank you

**Thank you for going through this notebook**

**If you have any suggestions please let me know**

<a id="5"></a>
# References
https://gist.github.com/karpathy/d4dee566867f8291f086

https://arxiv.org/abs/1711.05101

<div style="padding:10px;
            color:#333333;
            margin:10px;
            font-size:150%;
            display:fill;
            border-radius:1px;
            border-style:solid;
            border-color:#666666;
            background-color:#F9F9F9;
            overflow:hidden;">
    <center>
        <a id='top'></a>
        <b>Machine Learning From Scratch Series</b>
    </center>
    <br>
    <ul>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/linear-regression-from-scratch" style="color:#0072B2">1 - Linear Regression</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/logistic-regression-from-scratch" style="color:#0072B2">2 -  Logistic Regression</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/kmeans-from-scratch" style="color:#0072B2">3 - KMeans</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/decision-tree-classifier-from-scratch" style="color:#0072B2">4 - Decision Trees</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/random-forest-classifier-from-scratch" style="color:#0072B2">5 -  Random Forest</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/knn-from-scratch" style="color:#0072B2">6 - KNearestNeighbor</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/pca-from-scratch?scriptVersionId=121402593" style="color:#0072B2">7 - PCA</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/svm-from-scratch" style="color:#0072B2">8 - SVM</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/naive-bayes-from-scratch" style="color:#0072B2">9 - Naive Baye</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/optimized-neural-network-from-scratch" style="color:#0072B2">10 - Optimized Neural Network</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/neural-network-from-scratch" style="color:#0072B2">11 - Neural Network</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/cnn-from-scratch" style="color:#0072B2">12 - CNN</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/rnn-from-scratch" style="color:#0072B2">13 - RNN</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/lstm-from-scratch" style="color:#0072B2">14 - LSTM</a>
        </li>
        <li>
            <a href="https://www.kaggle.com/code/fareselmenshawii/gru-from-scratch" style="color:#0072B2">15 - GRU</a>
        </li>
    </ul>
</div>